In this notebook, we practice with data warehousing. We do have small warehouse with the same star schema as the lecture notes for `salesdb_sales`, `salesdb_item`, `salesdb_customer`, and `salesdb_store`.


**Submission Instruction**

1- Make a copy and replace blank with your name

2- Complete and run all cells

3- Download .ipynb and submit on Gradescope

In [ ]:
%pip install mysql-connector-python

In [ ]:
import mysql.connector
import pandas as pd

In [ ]:
mysql_address  = '131.193.32.85'
mysql_username='de_student'
mysql_password='DE_Student_PaSS'

mysql_database = 'my_dataengineering_dbs'


def get_conn_cur():
    cnx = mysql.connector.connect(user=mysql_username, password=mysql_password,
          host=mysql_address,
          database=mysql_database, port='3306');
    return (cnx, cnx.cursor())

def run_query(query_string):

  conn, cur = get_conn_cur() # get connection and cursor
  cur.execute(query_string) # executing string as before
  my_data = cur.fetchall() # fetch query data as before
  result_df = pd.DataFrame(my_data, columns=cur.column_names)


  cur.close() # close
  conn.close() # close

  return result_df


def sql_head(table_name):
  conn, cur = get_conn_cur() #get connection and cursor

  #get head information
  table_rows_query = """ SELECT * FROM %s LIMIT 5; """ % table_name
  cur.execute(table_rows_query)
  my_data = cur.fetchall() # fetch results

  # Create a dataframe that combines sql table with column names and return
  df = pd.DataFrame(my_data, columns=cur.column_names)

  cur.close()
  conn.close()
  return df

In [ ]:
sql_head("salesdb_customer")

In [ ]:
sql_head('salesdb_store')

In [ ]:
sql_head('salesdb_item')

In [ ]:
sql_head("salesdb_sales")

Now we look at couple of example queries.

 All inexpensive (price < 25) Tshirts sold in California to young people (age < 22)

In [ ]:
sql = """
SELECT S.city, I.color, C.cName, F.price
    FROM salesdb_sales F, salesdb_store S, salesdb_item I, salesdb_customer C
    WHERE F.storeID = S.storeID AND F.itemID = I.itemID
    AND F.custID = C.custID AND S.state = 'CA'
    AND I.category = 'Tshirt' AND C.age < 22 AND F.price < 25; """
run_query(sql)

Total sales by store ID and customer name

In [ ]:
sql= """
SELECT storeID, cName, sum(price)
FROM salesdb_sales S, salesdb_customer C
WHERE S.custID=C.custID
GROUP BY storeID, cName; """
run_query(sql)

"Drill-down" Total sales by store ID, category, and customer

In [ ]:
sql = """
SELECT storeID, I.category, cName, sum(price)
FROM salesdb_sales S, salesdb_customer C, salesdb_item I
WHERE S.custID=C.custID AND S.itemID=I.itemID
GROUP BY storeID, I.category, cName; """
run_query(sql)

"Slice" Total sales by store ID, category, and customer for "store6" only


In [ ]:
sql = """
SELECT S.storeID, I.category, cName, sum(price)
FROM salesdb_sales S, salesdb_customer C, salesdb_item I, salesdb_store T
WHERE S.custID=C.custID AND S.itemID=I.itemID
AND S.storeID=T.storeID AND T.storeId ='store6'
GROUP BY storeID, I.category, cName; """
run_query(sql)


"Dice" Total sales by store ID, category, and customer for "store6" and "Jacket" only

In [ ]:
sql = """
SELECT S.storeID, I.category, cName, sum(price)
FROM salesdb_sales S, salesdb_customer C, salesdb_item I, salesdb_store T
WHERE S.custID=C.custID AND S.itemID=I.itemID
AND S.storeID=T.storeID AND T.storeId ='store6' AND I.category='Jacket'
GROUP BY storeID, I.category, cName;"""

run_query(sql)

"Roll-up" Total sales by category

In [ ]:
sql = """
SELECT I.category, sum(price)
FROM salesdb_sales S, salesdb_item I
WHERE S.itemID=I.itemID
GROUP BY I.category;"""
run_query(sql)

Total sales by state, county, city

In [ ]:
sql = """
SELECT state, county, city, sum(price)
FROM salesdb_sales F, salesdb_store S
WHERE F.storeID = S.storeID
GROUP BY state, county, city; """
run_query(sql)

Total sales by state, county, city WITH ROLLUP. The ROLLUP generates multiple grouping sets based on the columns or expressions specified in the GROUP BY clause, here state, county, and city.


In [ ]:
sql = """
SELECT state, county, city, sum(price)
FROM salesdb_sales F, salesdb_store S
WHERE F.storeID = S.storeID
GROUP BY state, county, city WITH ROLLUP; """
run_query(sql)

Question 1: Write the query to list total sales by state of store and age of customer - [3 points]

In [ ]:
...


Question 2: Drill down to items by item color (on the basis of the previous query) - [3 points]  

In [ ]:
...


Question 3: Use "with rollup" with the previous query - [3 points]

In [ ]:
...


Question 4: Slice by listing only items with blue color (on the basis of query of Question 2) - [3 points]

In [ ]:
,,,


Question 5: Rollup total sales by customer age and item color (on the basis of query of Question 2)  -- the remaining dimensions are customer age and item color. [3 points]

In [ ]:
...
